

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/CLINICAL_CLASSIFICATION.ipynb)




# **How to use Licensed Classification models in Spark NLP**

### Spark NLP documentation and instructions:
https://nlp.johnsnowlabs.com/docs/en/quickstart

### You can find details about Spark NLP annotators here:
https://nlp.johnsnowlabs.com/docs/en/annotators

### You can find details about Spark NLP models here:
https://nlp.johnsnowlabs.com/models


To run this yourself, you will need to upload your license keys to the notebook. Otherwise, you can look at the example outputs at the bottom of the notebook. To upload license keys, open the file explorer on the left side of the screen and upload `workshop_license_keys.json` to the folder that opens.

## 1. Colab Setup

Import license keys

In [1]:
import os
import json

with open('/content/spark_nlp_for_healthcare.json', 'r') as f:
    license_keys = json.load(f)

license_keys.keys()

secret = license_keys['SECRET']
os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['AWS_ACCESS_KEY_ID'] = license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']
sparknlp_version = license_keys["PUBLIC_VERSION"]
jsl_version = license_keys["JSL_VERSION"]

print ('SparkNLP Version:', sparknlp_version)
print ('SparkNLP-JSL Version:', jsl_version)

SparkNLP Version: 2.6.3
SparkNLP-JSL Version: 2.7.0



Install dependencies

In [2]:
# Install Java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp==$sparknlp_version
! python -m pip install --upgrade spark-nlp-jsl==$jsl_version --extra-index-url https://pypi.johnsnowlabs.com/$secret

openjdk version "11.0.9" 2020-10-20
OpenJDK Runtime Environment (build 11.0.9+11-Ubuntu-0ubuntu1.18.04.1)
OpenJDK 64-Bit Server VM (build 11.0.9+11-Ubuntu-0ubuntu1.18.04.1, mixed mode, sharing)
     |████████████████████████████████| 215.7MB 58kB/s 
     |████████████████████████████████| 204kB 46.6MB/s 
     |████████████████████████████████| 133kB 2.8MB/s 
Looking in indexes: https://pypi.org/simple, https://pypi.johnsnowlabs.com/2.7.0-e49fd2919a73690cd04ed3b6e223e21330c5214b
     |████████████████████████████████| 51kB 372kB/s 


Import dependencies into Python and start the Spark session

In [2]:
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['PATH'] = os.environ['JAVA_HOME'] + "/bin:" + os.environ['PATH']

import pandas as pd
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import sparknlp
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl

spark = sparknlp_jsl.start(secret)

# manually start session
'''
builder = SparkSession.builder \
    .appName('Spark NLP Licensed') \
    .master('local[*]') \
    .config('spark.driver.memory', '16G') \
    .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer') \
    .config('spark.kryoserializer.buffer.max', '2000M') \
    .config('spark.jars.packages', 'com.johnsnowlabs.nlp:spark-nlp_2.11:' +sparknlp.version()) \
    .config('spark.jars', f'https://pypi.johnsnowlabs.com/{secret}/spark-nlp-jsl-{jsl_version}.jar')
'''

"\nbuilder = SparkSession.builder     .appName('Spark NLP Licensed')     .master('local[*]')     .config('spark.driver.memory', '16G')     .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')     .config('spark.kryoserializer.buffer.max', '2000M')     .config('spark.jars.packages', 'com.johnsnowlabs.nlp:spark-nlp_2.11:' +sparknlp.version())     .config('spark.jars', f'https://pypi.johnsnowlabs.com/{secret}/spark-nlp-jsl-{jsl_version}.jar')\n"

## 2. Usage Guidelines

1. **Selecting the correct Classification Model**

> a. To select from all the Classification models available in Spark NLP please go to https://nlp.johnsnowlabs.com/models

> b. Read through the model descriptions to select desired model

> c. Some of the available models:
>> classifierdl_pico_biobert

>> classifierdl_ade_biobert
---
2. **Selecting correct embeddings for the chosen model**

> a. Models are trained on specific embeddings and same embeddings should be used at inference to get best results

> b. If the name of the model contains "**biobert**" (e.g: *ner_anatomy_biobert*) then the model is trained using "**biobert_pubmed_base_cased**" embeddings. Otherwise, "**embeddings_clinical**" was used to train that model.

> c. Using correct embeddings

>> To use *embeddings_clinical* :

>>> word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

>> To use *Bert* Embeddings:

>>> embeddings = BertEmbeddings.pretrained('biobert_pubmed_base_cased')\
    .setInputCols(["document", 'token'])\
    .setOutputCol("word_embeddings")
> d. You can find list of all embeddings at https://nlp.johnsnowlabs.com/models?tag=embeddings


Create the pipeline

In [24]:
document_assembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")

embeddings = BertEmbeddings.pretrained('biobert_pubmed_base_cased')\
    .setInputCols(["document", 'token'])\
    .setOutputCol("word_embeddings")

sentence_embeddings = SentenceEmbeddings() \
      .setInputCols(["document", "word_embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE").setStorageRef('SentenceEmbeddings_5d018a59d7c3')

åå
classifier = ClassifierDLModel.pretrained('classifierdl_pico_biobert', 'en', 'clinical/models')\
    .setInputCols(['document', 'token', 'sentence_embeddings']).setOutputCol('class')

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    embeddings,
    sentence_embeddings, 
    classifier])

empty_data = spark.createDataFrame([[""]]).toDF("text")

pipeline_model = pipeline.fit(empty_data)

lmodel = LightPipeline(pipeline_model)

biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[OK!]
classifierdl_pico_biobert download started this may take some time.
Approximate size to download 22.1 MB
[OK!]


## 3. Create example inputs

In [26]:
# Enter examples as strings in this array
input_list = [
    """A total of 10 adult daily smokers who reported at least one stressful event and coping episode and provided post-quit data.""",
]

## 4. Use the pipeline to create outputs

Full Pipeline (Expects a Spark Data Frame)

In [27]:
df = spark.createDataFrame(pd.DataFrame({"text": input_list}))
result = pipeline_model.transform(df)

Light Pipeline (Expects a list of string

In [34]:
lresult = lmodel.fullAnnotate(input_list)

## 5. Visualize results

Full Pipeline Results

In [38]:
result.toPandas()['class'].iloc[0][0].result

'PARTICIPANTS'

Light Pipeline Results

In [36]:
lresult[0]['class'][0].result

'PARTICIPANTS'